<a href="https://colab.research.google.com/github/leonammeta8154/Estela_IA/blob/main/Estela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [14]:
!pip install -q google-adk

In [15]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [16]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é uma assistente de pesquisa, seu nome é Estela. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar opções de negócios mais relevantes sobre o tópico abaixo.
        Foque em no máximo 5 negócios, com base na quantidade e entusiasmo das notícias sobre eles.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses negócios relevantes devem ser atuais, de no máximo seis meses antes da data de hoje.
        """,
        description="Agente de pesquisa que busca informações no Google focado em identificar negócios relevantes para mulheres.",
        tools=[google_search]
    )

    entrada = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    negocios = call_agent(buscador, entrada)
    return negocios


In [17]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é uma Consultora empresarial, especialista em ajudar mulheres a começar seu próprio negócio. Seu nome é Estela.
        Com base na lista de negócios mais recentes e relevantes, você deve:
        - usar a ferramenta de busca do google (google_search) para criar um plano com os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles.
        - aprofundar-se nos temas usando (google_search) se necessário.
        Ao final, escolha o tema mais relevante entre eles com base nas suas pesquisas e retorne:
        - esse tema,
        - seus pontos mais relevantes,
        - um plano com os assuntos a serem abordados no post.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada = f"Tópico: {topico}\nLançamentos buscados: {lancamentos_buscados}"
    plano_do_post = call_agent(planejador, entrada)
    return plano_do_post


In [18]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Redator Criativo especializado em criar posts virais para redes sociais.
        Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva um rascunho de post para Instagram sobre o tema indicado.
        O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
        """,
        description="Agente redator de posts engajadores para Instagram"
    )

    entrada = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    rascunho = call_agent(redator, entrada)
    return rascunho


In [10]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    negocios_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(negocios_buscados))
    print("--------------------------------------------------------------")

In [23]:
from datetime import date
from IPython.display import display
from tabulate import tabulate
from pandas import DataFrame

# Exemplo de função simulada (substitua com sua lógica real)
def agente_buscador(topico, data):
    return DataFrame({
        "Negócio": ["Exemplo A", "Exemplo B"],
        "Tendência": ["Crescimento", "Declínio"]
    })

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀\n")

# --- Fazer 5 perguntas ---
perguntas = [
    "1️⃣ Qual é o seu nome?\n",
    "2️⃣ Qual é o seu principal objetivo agora, amiga? (Criar um negócio, fazer uma renda extra, mudar de carreira, outro?)\n",
    "3️⃣ Você já tem alguma ideia de negócio em mente?\n",
    "4️⃣ Você tem alguma experiência profissional ou área de formação? (Ex: marketing, direito, enfermagem...)\n",
    "5️⃣ Qual é a sua maior preocupação ou dúvida ao começar seu negócio?\n"
]

respostas = []

for pergunta in perguntas:
    resposta = input(pergunta + " ")
    respostas.append(resposta)

# --- Mostrar as respostas abaixo ---
#print("\n✅ Respostas recebidas:\n")
#for i in range(len(perguntas)):
    #print(f"{perguntas[i]} {respostas[i]}")

# --- Usar a primeira resposta como tópico principal ---
topico = respostas[0]

# --- Inserir lógica do sistema de agentes ---
if not topico:
    print("\n⚠️ Você esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}...")

    negocios_buscados = agente_buscador(topico, data_de_hoje)

    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(negocios_buscados)  # usa display para Jupyter, substitua por print(tabulate(...)) se for script
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀

1️⃣ Qual é o seu nome?
 maria
2️⃣ Qual é o seu principal objetivo agora, amiga? (Criar um negócio, fazer uma renda extra, mudar de carreira, outro?)
 renda extra
3️⃣ Você já tem alguma ideia de negócio em mente?
 vendas de roupas
4️⃣ Você tem alguma experiência profissional ou área de formação? (Ex: marketing, direito, enfermagem...)
 vendas
6️⃣ Qual é a sua maior preocupação ou dúvida ao começar seu negócio?
 não ter tempo

Maravilha! Vamos então criar o post sobre novidades em maria...

--- 📝 Resultado do Agente 1 (Buscador) ---



,Negócio,Tendência
0,Exemplo A,Crescimento
1,Exemplo B,Declínio


--------------------------------------------------------------


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Negócio').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('Tendência')):
  _plot_series(series, series_name, i)
  fig.legend(title='Tendência', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Tendência'].value_counts()
    for x_label, grp in _df_6.groupby('Negócio')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Negócio')
_ = plt.ylabel('Tendência')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['Negócio'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='index', y='Negócio', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['Tendência'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='Tendência', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)